In [78]:
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import random, math


In [79]:
path_age_data = './storage/age_data_2005.xls'
sheet2m = 'm; 2010-50, medium-fertility'
sheet2f = 'f; 2010-50, medium-fertility'

In [80]:
choosenCountryName = 'Kazakhstan'
choosenCountryCode = 398

In [81]:
cols = [
    'Index', 'Variant', 'Area', 'Notes',
    'Country_code', 'Reference_date',
    '0 - 4', '5 - 9', '10 - 14', '15 - 19',
    '20 - 24', '25 - 29', '30 - 34', '35 - 39',
    '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
    '65 - 69', '70 - 74', '75 - 79', '80 - 84', '85 - 89', '90 - 94',
    '95 - 99', '100+'
]
print(len(cols))
cols_ext_period = [ 'Reference_date', 'Sex',
    '0 - 4', '5 - 9', '10 - 14', '15 - 19',
    '20 - 24', '25 - 29', '30 - 34', '35 - 39',
    '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
    '65 - 69', '70 - 74', '75 - 79', '80 - 84'
    # ,'85 - 89', '90 - 94', '95 - 99', '100+'
]
print(len(cols_ext_period))
cols_period = [
    '0 - 4', '5 - 9', '10 - 14', '15 - 19',
    '20 - 24', '25 - 29', '30 - 34', '35 - 39',
    '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
    '65 - 69', '70 - 74', '75 - 79', '80 - 84'
    # ,'85 - 89', '90 - 94', '95 - 99', '100+'
]
print(len(cols_period))


27
19
17


# Getting data

In [82]:
dff = pd.read_excel(path_age_data, sheet_name=sheet2f)
dff.columns = cols
dff.dropna(subset=['Index', 'Variant', 'Area'], inplace=True)
dff = dff.iloc[1: , :]
dff['Sex'] = 'f'
dff

,Index,Variant,Area,Notes,Country_code,Reference_date,0 - 4,5 - 9,10 - 14,15 - 19,...,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 - 94,95 - 99,100+,Sex
6,0,Medium,WORLD,NaN,900,2010,308533.785,294657.693,290611.684,292432.301,...,122511.733,93635.052,78326.444,57164.141,38045.161,19800.176,7011.671,2016.854,321.32,f
7,0,Medium,WORLD,NaN,900,2015,315805.329,303018.991,292885.218,289573.031,...,148308.309,113789.754,82929.566,64238.548,41702.109,23263.384,9596.496,2454.44,462.89,f
8,0,Medium,WORLD,NaN,900,2020,315649.009,310982.69,301400.962,291929.141,...,162404.154,138320.502,101618.805,68871.533,47477.582,26057.288,11572.483,3504.545,610.63,f
9,0,Medium,WORLD,NaN,900,2025,310182.964,311508.632,309625.656,300538.936,...,185253.907,152021.971,124167.813,85350.823,51795.206,30199.413,13335.882,4375.202,889.925,f
10,0,Medium,WORLD,NaN,900,2030,304844.9,306661.641,310398.204,308865.187,...,204234.111,174135.457,137163.773,104977.138,65159.676,33746.3,15799.698,5229.787,1183.059,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,259,Medium,Tonga,NaN,776,2030,3.893,3.86,4.279,3.98,...,2.447,1.328,1.254,1.012,0.632,0.359,0.143,0.043,0.007,f
2027,259,Medium,Tonga,NaN,776,2035,3.419,3.483,3.896,3.6,...,2.103,1.881,1.34,1.054,0.761,0.397,0.175,0.048,0.009,f
2028,259,Medium,Tonga,NaN,776,2040,2.88,3.011,3.519,3.218,...,2.352,1.567,1.844,1.134,0.801,0.486,0.197,0.06,0.01,f
2029,259,Medium,Tonga,NaN,776,2045,2.379,2.473,3.047,2.841,...,3.062,1.807,1.568,1.571,0.871,0.519,0.247,0.069,0.013,f


In [83]:
dfm = pd.read_excel(path_age_data, sheet_name=sheet2m)
dfm.columns = cols
dfm.dropna(subset=['Index', 'Variant', 'Area'], inplace=True)
dfm = dfm.iloc[1: , :]
dfm['Sex'] = 'm'
dfm

,Index,Variant,Area,Notes,Country_code,Reference_date,0 - 4,5 - 9,10 - 14,15 - 19,...,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 - 94,95 - 99,100+,Sex
6,0,Medium,WORLD,NaN,900,2010,324453.051,310015.285,306138.861,308097.76,...,115546.475,83920.135,64827.917,43392.716,24623.641,10130.297,2814.41,579.507,70.612,m
7,0,Medium,WORLD,NaN,900,2015,332039.152,318537.914,308006.748,304640.396,...,140169.446,102584.743,69794.857,48624.43,27908.199,12593.943,3801.933,710.932,96.705,m
8,0,Medium,WORLD,NaN,900,2020,331693.21,326749.369,316675.114,306600.863,...,154172.517,125079.555,86125.931,53234.511,31788.912,14568.028,4848.809,984.569,122.498,m
9,0,Medium,WORLD,NaN,900,2025,325721.711,327066.55,325143.53,315364.115,...,175198.142,138300.977,105630.162,66460.298,35498.631,16928.036,5737.2,1290.96,168.187,m
10,0,Medium,WORLD,NaN,900,2030,319583.011,321714.812,325704.853,323932.697,...,194019.158,158169.25,117664.376,82109.358,45013.187,19405.719,6846.228,1568.629,224.869,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,259,Medium,Tonga,NaN,776,2030,4.067,4.148,4.404,4.259,...,2.627,1.814,1.265,0.955,0.504,0.245,0.086,0.016,0.002,m
2027,259,Medium,Tonga,NaN,776,2035,3.567,3.748,3.995,3.855,...,2.204,2.38,1.54,0.964,0.63,0.262,0.091,0.02,0.002,m
2028,259,Medium,Tonga,NaN,776,2040,2.997,3.249,3.596,3.447,...,2.524,2.015,2.032,1.188,0.645,0.335,0.1,0.022,0.002,m
2029,259,Medium,Tonga,NaN,776,2045,2.467,2.68,3.097,3.048,...,3.182,2.316,1.738,1.585,0.807,0.35,0.131,0.024,0.003,m


In [84]:
df = dfm.append(dff)
df

,Index,Variant,Area,Notes,Country_code,Reference_date,0 - 4,5 - 9,10 - 14,15 - 19,...,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 - 94,95 - 99,100+,Sex
6,0,Medium,WORLD,NaN,900,2010,324453.051,310015.285,306138.861,308097.76,...,115546.475,83920.135,64827.917,43392.716,24623.641,10130.297,2814.41,579.507,70.612,m
7,0,Medium,WORLD,NaN,900,2015,332039.152,318537.914,308006.748,304640.396,...,140169.446,102584.743,69794.857,48624.43,27908.199,12593.943,3801.933,710.932,96.705,m
8,0,Medium,WORLD,NaN,900,2020,331693.21,326749.369,316675.114,306600.863,...,154172.517,125079.555,86125.931,53234.511,31788.912,14568.028,4848.809,984.569,122.498,m
9,0,Medium,WORLD,NaN,900,2025,325721.711,327066.55,325143.53,315364.115,...,175198.142,138300.977,105630.162,66460.298,35498.631,16928.036,5737.2,1290.96,168.187,m
10,0,Medium,WORLD,NaN,900,2030,319583.011,321714.812,325704.853,323932.697,...,194019.158,158169.25,117664.376,82109.358,45013.187,19405.719,6846.228,1568.629,224.869,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,259,Medium,Tonga,NaN,776,2030,3.893,3.86,4.279,3.98,...,2.447,1.328,1.254,1.012,0.632,0.359,0.143,0.043,0.007,f
2027,259,Medium,Tonga,NaN,776,2035,3.419,3.483,3.896,3.6,...,2.103,1.881,1.34,1.054,0.761,0.397,0.175,0.048,0.009,f
2028,259,Medium,Tonga,NaN,776,2040,2.88,3.011,3.519,3.218,...,2.352,1.567,1.844,1.134,0.801,0.486,0.197,0.06,0.01,f
2029,259,Medium,Tonga,NaN,776,2045,2.379,2.473,3.047,2.841,...,3.062,1.807,1.568,1.571,0.871,0.519,0.247,0.069,0.013,f


In [85]:
d_splited = df[ df['Country_code']==choosenCountryCode ][cols_ext_period]
d_splited

,Reference_date,Sex,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84
744,2010,m,551.44,527.118,532.465,620.22,778.31,637.738,567.77,523.364,432.198,503.798,425.954,351.155,184.392,131.194,168.297,64.555,48.55
745,2015,m,560.964,535.575,510.62,513.625,598.343,753.341,612.803,539.539,493.586,402.865,462.056,379.858,297.755,146.689,96.529,110.456,35.265
746,2020,m,529.328,545.992,519.458,492.345,493.513,577.038,727.52,584.873,510.888,463.401,370.099,415.447,325.547,241.842,110.007,64.368,62.016
747,2025,m,477.05,515.627,530.16,501.498,473.038,474.543,555.616,699.145,556.856,482.175,429.368,334.252,359.579,267.264,184.495,74.449,36.549
748,2030,m,432.262,464.478,500.129,512.439,482.632,455.06,455.637,531.765,670.298,528.565,449.493,391.097,290.714,298.047,206.016,126.831,42.949
749,2035,m,412.229,420.541,449.313,482.758,493.958,465.22,437.215,434.351,507.883,640.501,495.845,412.113,343.448,242.355,232.052,143.335,74.471
750,2040,m,407.092,401.026,405.625,432.285,464.814,477.01,447.946,416.953,413.271,484.482,605.008,457.594,364.654,289.113,190.109,163.421,85.416
751,2045,m,396.271,396.252,386.269,388.865,414.888,448.552,460.229,428.237,396.955,393.521,457.616,562.101,407.925,309.355,229.04,135.191,98.719
752,2050,m,370.329,385.729,381.596,369.668,371.874,399.296,432.476,440.966,408.75,378.515,371.615,425.973,505.132,348.904,247.346,164.874,82.751
744,2010,f,531.77,511.709,514.952,599.523,765.301,635.753,597.109,546.17,468.756,571.169,504.985,453.326,251.553,210.912,279.557,117.081,122.937


In [86]:
dmal = d_splited[ d_splited['Sex']=='m' ]
print(dmal)
dfem = d_splited[ d_splited['Sex']=='f' ]
print(dfem)

    Reference_date Sex    0 - 4    5 - 9  10 - 14  15 - 19  20 - 24  25 - 29  \
744           2010   m   551.44  527.118  532.465   620.22   778.31  637.738   
745           2015   m  560.964  535.575   510.62  513.625  598.343  753.341   
746           2020   m  529.328  545.992  519.458  492.345  493.513  577.038   
747           2025   m   477.05  515.627   530.16  501.498  473.038  474.543   
748           2030   m  432.262  464.478  500.129  512.439  482.632   455.06   
749           2035   m  412.229  420.541  449.313  482.758  493.958   465.22   
750           2040   m  407.092  401.026  405.625  432.285  464.814   477.01   
751           2045   m  396.271  396.252  386.269  388.865  414.888  448.552   
752           2050   m  370.329  385.729  381.596  369.668  371.874  399.296   

     30 - 34  35 - 39  40 - 44  45 - 49  50 - 54  55 - 59  60 - 64  65 - 69  \
744   567.77  523.364  432.198  503.798  425.954  351.155  184.392  131.194   
745  612.803  539.539  493.586  402.865  

In [87]:
db = d_splited.groupby(by=['Reference_date'])[cols_period].sum()
db.insert(0, 'Reference_date', db.index)
db.reset_index(drop=True, inplace=True)
db

,Reference_date,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84
0,2010,1083.210,1038.827,1047.417,1219.743,1543.611,1273.491,1164.879,1069.534,900.954,1074.967,930.939,804.481,435.945,342.106,447.854,181.636,171.487
1,2015,1100.928,1054.388,1006.961,1011.618,1181.631,1504.670,1233.877,1118.677,1021.945,855.495,1011.310,860.484,718.314,370.612,273.860,327.761,113.283
2,2020,1038.260,1073.427,1023.033,971.948,976.057,1147.780,1464.101,1188.397,1072.563,975.470,804.889,940.205,773.654,621.453,300.009,202.734,210.468
3,2025,934.469,1012.770,1042.493,988.499,937.593,945.367,1112.908,1417.998,1143.286,1027.823,923.124,749.692,851.065,673.955,511.623,224.448,131.866
4,2030,845.448,910.757,982.327,1008.322,954.812,908.228,913.703,1072.727,1371.664,1099.323,976.985,864.592,679.873,746.537,558.950,389.687,147.902
5,2035,805.197,823.077,880.799,948.670,975.216,926.220,877.911,876.863,1033.163,1325.470,1048.772,919.305,789.062,597.912,623.824,429.599,261.922
6,2040,794.318,783.622,793.470,847.619,916.296,947.196,896.578,842.336,841.036,995.879,1270.840,990.439,843.677,698.542,501.647,484.327,292.428
7,2045,772.460,773.278,754.231,760.653,815.974,889.124,918.108,861.639,807.879,808.765,953.509,1205.751,912.682,751.438,590.390,391.894,333.865
8,2050,721.326,751.882,744.025,721.630,729.558,789.661,860.903,883.688,827.808,777.350,773.366,904.831,1117.383,816.533,639.791,465.271,272.602


In [88]:
dp = pd.DataFrame(db)
dp['Popularity'] = dp[cols_period].sum(axis=1)
dp = dp[['Reference_date', 'Popularity']]
dp

,Reference_date,Popularity
0,2010,14731.081
1,2015,14765.814
2,2020,14784.448
3,2025,14628.979
4,2030,14431.837
5,2035,14142.982
6,2040,13740.250
7,2045,13301.640
8,2050,12797.608


In [89]:
d_surv_rate_m = pd.read_csv('./storage/intermediante/surv_rate_m.csv', index_col='index')
d_surv_rate_len = len(d_surv_rate_m.columns)
print(d_surv_rate_len)
d_surv_rate_m.values[0]

16


array([0.97331023, 0.96990711, 0.96421018, 0.96084303, 0.96439333,
       0.96572469, 0.95442255, 0.94689726, 0.93884551, 0.91866754,
       0.89912695, 0.85702289, 0.8122181 , 0.74993353, 0.66682551,
       0.56145433])

In [90]:
d_surv_rate_f = pd.read_csv('./storage/intermediante/surv_rate_f.csv', index_col='index')
d_surv_rate_f

,0 - 4/5 - 9,5 - 9/10 - 14,10 - 14/15 - 19,15 - 19/20 - 24,20 - 24/25 - 29,25 - 29/30 - 34,30 - 34/35 - 39,35 - 39/40 - 44,40 - 44/45 - 49,45 - 49/50 - 54,50 - 54/55 - 59,55 - 59/60 - 64,60 - 64/65 - 69,65 - 69/70 - 74,70 - 74/75 - 79,75 - 79/80 - 84
index,,,,,,,,,,,,,,,,
0,0.97331,0.969907,0.96421,0.960843,0.964393,0.965725,0.954423,0.946897,0.938846,0.918668,0.899127,0.857023,0.812218,0.749934,0.666826,0.561454


# Model

In [91]:
def Model(coef_fertility, coef_bg_ratio, survival_rate):
    return coef_fertility*(coef_bg_ratio+survival_rate)

In [92]:
# args - 
prev_record_m = dmal.iloc[-1].values
prev_record_f = dfem.iloc[-1].values
# surv_rate_m = d_surv_rate_m.values[0]
# surv_rate_f = d_surv_rate_f.values[0]

# args - 
def Model(args):

    coef_fertility = args[0]
    coef_bg_ratio_m = args[1]
    coef_bg_ratio_f = args[2]
    surv_rate_m = args[3:3+d_surv_rate_len]
    surv_rate_f = args[3+d_surv_rate_len:]

    # res = pd.DataFrame([], columns=model_cols)
    # res.Reference_date = start_year
    
    resm = []
    resf = []
    
    resm.append(prev_record_m[0]+5)
    resf.append(prev_record_f[0]+5)

    resm.append('m')
    resf.append('f')

    tmpm = []
    for prev_period, rate in zip(prev_record_m[2:], surv_rate_m):
        alive_predict_m = (prev_period * rate)
        
        tmpm.append(alive_predict_m)
        continue
    # print('len(tmpm)', len(tmpm))

    tmpf = []
    for prev_period, rate in zip(prev_record_f[2:], surv_rate_f):
        alive_predict_f = (prev_period * rate)
        
        tmpf.append(alive_predict_f)
        continue
    # print('len(tmpf)', len(tmpf))

    women_amount = sum(tmpf[5:9])

    birthsm = (women_amount * coef_fertility) * coef_bg_ratio_m
    resm.append(birthsm)

    birthsf = (women_amount * coef_fertility) * coef_bg_ratio_f
    resf.append(birthsf)

    [resm.append(i) for i in tmpm]
    [resf.append(i) for i in tmpf]

    return resm, resf

Usfull links:
- https://salib.readthedocs.io/en/latest/basics.html

# 1 task

Perform a **sensitivity analysis** for a demographic model with respect to a set of parameters: fertility rate, boys/girls ratio, «survival» rate for different age groups (not all can be taken). Model output: number of inhabitants for a given year. Test on the final forecast values ​​for 10, 20, 50, 100 years.

In [93]:
def Evaluate(param_values):
    Y = []
    for params in param_values:
        # x1, x2, x3 = params
        # output = Model(x1, x2, x3)
        output = Model(params)
        
        # print(cols_ext_period)
        # print(len(output[0]), output[0])
        # print(len(output[1]), output[1])

        pred_male = pd.DataFrame([output[0]], columns=cols_ext_period)
        pred_female = pd.DataFrame([output[1]], columns=cols_ext_period)        

        pred = pd.DataFrame([], columns=cols_ext_period)
        pred = pred.append(pred_female)
        pred = pred.append(pred_male)
        pred_both = pred.groupby(by=['Reference_date'])[cols_period].sum()
        # pred_both.insert(0, 'Reference_date', pred_both.index)
        # pred_both.reset_index(drop=True, inplace=True)
        # print(pred_both)
        res = pred_both[cols_period].sum(axis=1)

        print(res.values[0])

        if res.values[0] in Y:
            print('repeat value ', res.values[0])
            # input()
            pass

        # Y.append(res[0])
        Y.append(res.values[0])
        
        continue
    return np.array(Y)

In [94]:
def problem_gen(vals, keys=None, print_res=False):
    problem = {
        'num_vars': len(vals)
    }

    if keys == None:
        keys = [f"x{i}" for i in range(1,len(vals)+1)]
        print('Keys setted as default:', keys)
        pass

    tmp_names = []
    tmp_bounds = []
    for key, val in zip(keys, vals):
        tmp_names.append(key)
        tmp_bounds.append(val)
        continue
        
    problem['names'] = tmp_names
    problem['bounds'] = tmp_bounds

    if print_res:
        print(problem)

    return problem

In [95]:
flat_vector = lambda min, max, size: [[min, max] for _ in range(0, size)]

# Define the model inputs
problem = problem_gen(
    vals=[
        [0.0, 1.0] # coef_fertility
        ,[0.0, 1.0] # coef_bg_ratio_m
        ,[0.0, 1.0] # coef_bg_ratio_f
        ,*flat_vector(0.0, 1.0, d_surv_rate_len) # flatted vector of survival rates male
        ,*flat_vector(0.0, 1.0, d_surv_rate_len) # flatted vector of survival rates female
    ],
    print_res=True
)


Keys setted as default: ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35']
{'num_vars': 35, 'names': ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35'], 'bounds': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]}


In [96]:
# Generate samples
param_values = saltelli.sample(problem, 1000)

# Run model (example)
Y = Evaluate(param_values)
print('evaluate passed')

# Perform analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# Print the first-order sensitivity indices
print("__________________")
print(Si['S1'])


c:\Unic-ITMO\DiscreteModeling\DiscModel_labs\.venv\lib\site-packages\SALib\sample\saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000) is equal to `2^n`.
        
  warnings.warn(msg)


7325.742671105369
7924.551391905742
7326.493072250801
7325.744224730928
7231.352174035056
7158.869675988181
7163.638901574119
7266.899034386618
7404.548001183493
7403.730171105368
7211.710913292868
7009.228989464744
7013.591792199119
7065.513608605369
7567.800488488181
7302.031283410056
7148.157202355368
7292.692194542868
7320.187048058493
7389.019508995993
7319.572801964744
7297.136967980368
7522.176357628807
7219.8791007928685
7192.309772667868
7159.33189940772
7264.584602491321
7368.59657305273
7264.283108061096
7585.530703331931
7452.466863488181
6959.584649620993
7319.763657433494
7369.582889855368
7386.295708214743
7490.608993370994
6023.234549853139
6486.445196244049
6751.577574465877
6846.518138710923
6919.000636757798
6914.231411171861
6810.971278359361
6673.322311562486
6674.140141640611
6866.159399453111
7068.641323281236
7064.278520546861
7012.35670414061
6510.069824257798
6775.839029335923
6929.713110390611
6785.178118203111
6757.6832646874855
6688.850803749986
6758.297510

# 2 task

Define ranges of model parameter values ​​from data for previous periods (1950-2000)

In [97]:
Model(Si['S1'])

([2055,
  'm',
  0.22201805294185373,
  6.333971398038957,
  6.1568308389850674,
  8.132989478728373,
  6.746544082840789,
  8.296913029157938,
  9.814970927134187,
  11.505818334307836,
  12.497131221307725,
  8.58493294793238,
  5.217981067455636,
  7.919257171157002,
  10.602362464814922,
  20.36368683065752,
  5.437341071988621,
  2.4265419201655485,
  0.8707210020213749],
 [2055,
  'f',
  0.20408062767348242,
  6.046457671814466,
  7.054318152494141,
  5.816229249798537,
  5.858473906903842,
  6.6390944952292035,
  21.451754651117703,
  26.51388964999971,
  25.787337689320374,
  23.262507986394457,
  10.112549428987187,
  9.212848598094705,
  16.005278601111247,
  32.009968497365286,
  14.12530541724076,
  9.830563671756595,
  3.5423468714774757])

# 3 task

Based on all ranges of parameter values, perform an **uncertainty analysis** in the form of a graph with confidence intervals of the results. The values ​​between the boundaries can be considered evenly distributed.